This tutorial explains some basic RDD transformations and actions.

In [2]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v2.4.3 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [3]:
#first create an RDD
data = sc.parallelize([1,2,3,3])
#data.take(4)

In [4]:
mapped_data = data.map(lambda x: x * x)

In [5]:
filtered_data = mapped_data.filter(lambda x: x > 2)

In [6]:
filtered_data.take(4)

Out[ 7 ]: [4, 9, 9]

In [7]:
#let us use a map transformation - use lambda function to return the cube of each element
data.map(lambda x: x * x * x).take(4)

Out[ 2 ]: [1, 8, 27, 27]

In [8]:
#use a function instead of a lambda function - we will get the square root of each element
import math
def convert(x):
    return math.sqrt(x)
data.map(convert).take(4)

In [9]:
#use filter to get only odd numbers
data.filter(lambda x: x % 2 != 0).take(4) #there will be just 3 odd elements

In [10]:
#let us remove duplicates
data.distinct().take(4)

Out[ 3 ]: [1, 2, 3]

In [11]:
#let us sample just two elements
data.sample(False, 0.2).take(4) #False implies no replacement; 0.2 is the probability of each element being chosen

Out[ 4 ]: [1, 3]

In [12]:
from __future__ import print_function
#let us try a few set-like RDD operations
rdd1 = sc.parallelize([1,2,3,4,5])
rdd2 = sc.parallelize([3,5,6,7,8])
#union
rdd1.union(rdd2).collect()


Out[1]: [1, 2, 3, 4, 5, 3, 5, 6, 7, 8]

In [13]:
#intersection
rdd1.intersection(rdd2).collect()

Out[2]: [3, 5]

In [14]:
#subtract
rdd1.subtract(rdd2).collect()

Out[3]: [1, 2, 4]

In [15]:
#cartesian product
rdd1.cartesian(rdd2).collect()

Out[4]: [(1, 3),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (2, 3),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (3, 3),
 (3, 5),
 (3, 6),
 (3, 7),
 (3, 8),
 (4, 3),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 8),
 (5, 3),
 (5, 5),
 (5, 6),
 (5, 7),
 (5, 8)]

In [16]:
#let us look at some actions
rdd1.collect() #returns all the elements from the RDD

In [17]:
#number of elements in the RDD
rdd1.count()

In [18]:
#Let us create a union and check to see how many times each element occurs in the RDD
u = rdd1.union(rdd2)
u.countByValue()

Out[ 6 ]: defaultdict(<class 'int'>, {1: 1, 2: 1, 3: 2, 4: 1, 5: 2, 6: 1, 7: 1, 8: 1})

In [19]:
#return the first element
u.first()

In [20]:
#return the first 3 elements
u.take(3)

In [21]:
#return the top n elements
u.top(3)

In [22]:
#rdd1 is [1,2,3,4,5]. Let us get the product of the elements
product = rdd1.reduce(lambda x, y: x * y)
print(product)

In [23]:
#let us try a few transformations on paired RDDs
#assume that we have multiple sales values per day....not all days of the week are shown
paired_RDD = sc.parallelize([("Mon", 200.00), ("Tue", 1215.50), ("Mon", 300.25),("Wed", 100.00),("Mon", 100.00)])
#get total sales by day
sales_by_day = paired_RDD.reduceByKey(lambda x, y: x + y)
sales_by_day.collect()
#paired_RDD.reduceByKey(lambda x, y: x + y)

Out[5]: [('Wed', 100.0), ('Mon', 600.25), ('Tue', 1215.5)]

In [24]:
#let us group by key - get key and an iterable of items
grouped_RDD = paired_RDD.groupByKey()
grouped_RDD.collect() #note that the value will be an iterator

Out[6]: [('Wed', <pyspark.resultiterable.ResultIterable at 0x7fb91828bf50>),
 ('Mon', <pyspark.resultiterable.ResultIterable at 0x7fb91828bfd0>),
 ('Tue', <pyspark.resultiterable.ResultIterable at 0x7fb91828be50>)]

In [25]:
#let us get a list instead of an iterator ...
list_RDD = paired_RDD.groupByKey().mapValues(list)
list_RDD.collect()

Out[7]: [('Wed', [100.0]), ('Mon', [200.0, 300.25, 100.0]), ('Tue', [1215.5])]

In [26]:
#let us convert list_RDD back to the paired RDD that we started with
original_RDD = list_RDD.flatMapValues(lambda x: x)
original_RDD.collect()

Out[8]: [('Wed', 100.0),
 ('Mon', 200.0),
 ('Mon', 300.25),
 ('Mon', 100.0),
 ('Tue', 1215.5)]

In [27]:
#let us sort out original RDD by key in descending order
sorted_RDD = original_RDD.sortByKey(ascending = False) #default is in ascending order
sorted_RDD.collect()

Out[9]: [('Wed', 100.0),
 ('Tue', 1215.5),
 ('Mon', 200.0),
 ('Mon', 300.25),
 ('Mon', 100.0)]

In [28]:
#let us sort by value
sort_by_value = original_RDD.sortBy(lambda item: item[1]) #use - item[1] to sort in descending order
sort_by_value.collect()

Out[11]: [('Wed', 100.0),
 ('Mon', 100.0),
 ('Mon', 200.0),
 ('Mon', 300.25),
 ('Tue', 1215.5)]

In [29]:
#simple word count
words = ["apple","mango","orange","apple","mango","mango","peach", "mango","guava", "apple"]
words_rdd = sc.parallelize(words)
words_rdd.take(5)

Out[12]: ['apple', 'mango', 'orange', 'apple', 'mango']

In [30]:
words_rdd.map(lambda word: (word, 1)).countByKey() #note that countByKey() is an action

Out[13]: defaultdict(int, {'apple': 3, 'mango': 4, 'orange': 1, 'peach': 1, 'guava': 1})

In [31]:
words_rdd.map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y).collect()

Out[16]: [('mango', 4), ('orange', 1), ('peach', 1), ('apple', 3), ('guava', 1)]

In [32]:
words_rdd.map(lambda word: (word, 1)).groupByKey().collect()

Out[17]: [('mango', <pyspark.resultiterable.ResultIterable at 0x7fb9182349d0>),
 ('orange', <pyspark.resultiterable.ResultIterable at 0x7fb9182340d0>),
 ('peach', <pyspark.resultiterable.ResultIterable at 0x7fb918234fd0>),
 ('apple', <pyspark.resultiterable.ResultIterable at 0x7fb918234a90>),
 ('guava', <pyspark.resultiterable.ResultIterable at 0x7fb918234310>)]

In [33]:
#let us repeat the previous one followed by mapValues to change the iterable to a list
words_rdd.map(lambda word: (word, 1)).groupByKey().mapValues(lambda v: list(v)).collect()

Out[18]: [('mango', [1, 1, 1, 1]),
 ('orange', [1]),
 ('peach', [1]),
 ('apple', [1, 1, 1]),
 ('guava', [1])]

In [34]:
#let us modify the previous statement to get the sum of the 1s in the list
words_rdd.map(lambda word: (word, 1)).groupByKey().mapValues(lambda v: sum(list(v))).collect()

Out[19]: [('mango', 4), ('orange', 1), ('peach', 1), ('apple', 3), ('guava', 1)]